#video classification (1) of COVID-US (data preparation)

https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/

**preprocessed(clean) videos -> split -> pre-trained CNN** -> LSTM or GRU -> dense layer -> result(accuracy)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/COVID-US-main/COVID-US-main

/content/drive/MyDrive/COVID-US-main/COVID-US-main


In [ ]:
route = '/content/drive/MyDrive/COVID-US-main/COVID-US-main/'

##0. Import libraries

In [ ]:
import cv2 # for capturing videos
import math # for mathematical operations
%matplotlib inline
import matplotlib.pyplot as plt # for plotting the images
import numpy as np # for mathematical operations
import pandas as pd
import tensorflow as tf

from skimage.transform import resize # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

import keras
from keras.utils import np_utils
from keras.preprocessing import image # for preprocessing the images
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import TimeDistributed, GRU, LSTM, GlobalAveragePooling1D

from scipy import stats as s
from sklearn.metrics import accuracy_score

import os
import time
import random
import PIL
from PIL import Image
# import re
# import shutil

In [ ]:
frames=14
width=224
height=224
channels=3
train_test_ratio=0.3
train_val_ratio=0.2

##1. Prepare preprocessd(clean) videos

In [ ]:
video_dataset = pd.read_csv("utils/video_files_clean_properties_addition.csv")
video_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,1_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,64.0,4.27,0,1
1,2_butterfly_covid_prc_convex_clean.avi,15.0,624.0,624.0,158.0,10.53,0,1
2,3_butterfly_covid_prc_convex_clean.avi,15.0,1054.0,1054.0,90.0,6.00,0,1
3,4_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,108.0,7.20,0,1
4,5_butterfly_covid_prc_convex_clean.avi,15.0,810.0,810.0,249.0,16.60,0,1
...,...,...,...,...,...,...,...,...
237,239_paper_normal_prc_convex_clean.avi,15.0,512.0,512.0,197.0,13.13,2,8
238,240_paper_other_prc_convex_clean.avi,15.0,512.0,512.0,96.0,6.40,2,8
239,241_paper_other_prc_convex_clean.avi,15.0,512.0,512.0,98.0,6.53,2,8
240,242_paper_other_prc_convex_clean.avi,15.0,512.0,512.0,299.0,19.93,2,8


In [ ]:
video_dataset=video_dataset[video_dataset['filename'].str.contains('covid|pneumonia|normal')]
video_dataset=video_dataset[video_dataset['filename'].str.contains('convex')]

video_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,1_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,64.0,4.27,0,1
1,2_butterfly_covid_prc_convex_clean.avi,15.0,624.0,624.0,158.0,10.53,0,1
2,3_butterfly_covid_prc_convex_clean.avi,15.0,1054.0,1054.0,90.0,6.00,0,1
3,4_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,108.0,7.20,0,1
4,5_butterfly_covid_prc_convex_clean.avi,15.0,810.0,810.0,249.0,16.60,0,1
...,...,...,...,...,...,...,...,...
231,233_clarius_pneumonia_prc_convex_clean.avi,15.0,458.0,458.0,482.0,32.13,1,9
232,234_clarius_pneumonia_prc_convex_clean.avi,15.0,458.0,458.0,482.0,32.13,1,9
233,235_clarius_pneumonia_prc_convex_clean.avi,15.0,458.0,458.0,481.0,32.07,1,9
234,236_clarius_normal_prc_convex_clean.avi,15.0,482.0,482.0,365.0,24.33,2,9


In [ ]:
normal_dataset = video_dataset[video_dataset['filename'].str.contains('normal')]
normal_dataset.reset_index(drop = True)
normal_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
19,20_butterfly_normal_prc_convex_clean.avi,15.0,594.0,594.0,142.0,9.47,2,1
20,21_butterfly_normal_prc_convex_clean.avi,15.0,820.0,820.0,99.0,6.60,2,1
34,36_grepmed_normal_prc_convex_clean.avi,15.0,540.0,540.0,85.0,5.67,2,2
60,62_pocusatlas_normal_prc_convex_clean.avi,15.0,376.0,376.0,60.0,4.00,2,4
69,71_pocusatlas_normal_prc_convex_clean.avi,15.0,450.0,450.0,59.0,3.93,2,4
74,76_pocusatlas_normal_prc_convex_clean.avi,15.0,338.0,338.0,60.0,4.00,2,4
75,77_pocusatlas_normal_prc_convex_clean.avi,15.0,236.0,236.0,60.0,4.00,2,4
150,152_radio_normal_prc_convex_clean.avi,15.0,270.0,270.0,131.0,8.73,2,5
173,175_uf_normal_prc_convex_clean.avi,15.0,480.0,480.0,38.0,2.53,2,7
174,176_uf_normal_prc_convex_clean.avi,15.0,512.0,512.0,53.0,3.53,2,7


In [10]:
import shutil
shutil.copytree("data/video/clean", "data/video/clean_copy2")

'data/video/clean_copy2'

In [11]:
aug_file_list = []
for each_video in normal_dataset['filename']:
    cap = cv2.VideoCapture('data/video/clean/' + each_video)
    frameFPS = cap.get(cv2.CAP_PROP_FPS)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"DIVX") # codec
    filename = 'data/video/clean_copy2/' + each_video
    index = filename.find('.avi')
    aug_filename = filename[:index] + '_aug' + filename[index:]
    print(aug_filename)
    aug_file_list.append(aug_filename.split('/')[-1])
    videoWriter = cv2.VideoWriter(aug_filename, fourcc, frameFPS, (frameWidth, frameHeight))

    while (cap.isOpened()):
        ret, frame = cap.read()
        frame = cv2.flip(frame,1)
        if frame is None:
            break
        videoWriter.write(frame)
  
    cap.release()
    videoWriter.release()

data/video/clean_copy2/20_butterfly_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/21_butterfly_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/36_grepmed_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/62_pocusatlas_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/71_pocusatlas_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/76_pocusatlas_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/77_pocusatlas_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/152_radio_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/175_uf_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/176_uf_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/177_uf_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/180_uf_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/190_uf_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/191_uf_normal_prc_convex_clean_aug.avi
data/video/clean_copy2/206_paper_normal_prc_convex_clean_aug.avi
data/video/clean_cop

In [12]:
aug_dataset = pd.DataFrame()
aug_dataset['filename'] = aug_file_list
aug_dataset['framerate'] = normal_dataset['framerate'].tolist()
aug_dataset['width'] = normal_dataset['width'].tolist()
aug_dataset['height'] = normal_dataset['height'].tolist()
aug_dataset['frame_count'] = normal_dataset['frame_count'].tolist()
aug_dataset['duration_secs'] = normal_dataset['duration_secs'].tolist()
aug_dataset['disease'] = normal_dataset['disease'].tolist()
aug_dataset['source'] = normal_dataset['source'].tolist()
aug_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,20_butterfly_normal_prc_convex_clean_aug.avi,15.0,594.0,594.0,142.0,9.47,2,1
1,21_butterfly_normal_prc_convex_clean_aug.avi,15.0,820.0,820.0,99.0,6.60,2,1
2,36_grepmed_normal_prc_convex_clean_aug.avi,15.0,540.0,540.0,85.0,5.67,2,2
3,62_pocusatlas_normal_prc_convex_clean_aug.avi,15.0,376.0,376.0,60.0,4.00,2,4
4,71_pocusatlas_normal_prc_convex_clean_aug.avi,15.0,450.0,450.0,59.0,3.93,2,4
5,76_pocusatlas_normal_prc_convex_clean_aug.avi,15.0,338.0,338.0,60.0,4.00,2,4
6,77_pocusatlas_normal_prc_convex_clean_aug.avi,15.0,236.0,236.0,60.0,4.00,2,4
7,152_radio_normal_prc_convex_clean_aug.avi,15.0,270.0,270.0,131.0,8.73,2,5
8,175_uf_normal_prc_convex_clean_aug.avi,15.0,480.0,480.0,38.0,2.53,2,7
9,176_uf_normal_prc_convex_clean_aug.avi,15.0,512.0,512.0,53.0,3.53,2,7


In [13]:
video_dataset_copy = pd.concat([video_dataset, aug_dataset],ignore_index = True)
video_dataset_copy.reset_index(drop=True)
video_dataset = video_dataset_copy
video_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,1_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,64.0,4.27,0,1
1,2_butterfly_covid_prc_convex_clean.avi,15.0,624.0,624.0,158.0,10.53,0,1
2,3_butterfly_covid_prc_convex_clean.avi,15.0,1054.0,1054.0,90.0,6.00,0,1
3,4_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,108.0,7.20,0,1
4,5_butterfly_covid_prc_convex_clean.avi,15.0,810.0,810.0,249.0,16.60,0,1
...,...,...,...,...,...,...,...,...
136,206_paper_normal_prc_convex_clean_aug.avi,15.0,600.0,600.0,120.0,8.00,2,8
137,217_clarius_normal_prc_convex_clean_aug.avi,15.0,886.0,886.0,241.0,16.07,2,9
138,224_clarius_normal_prc_convex_clean_aug.avi,15.0,436.0,436.0,74.0,4.93,2,9
139,236_clarius_normal_prc_convex_clean_aug.avi,15.0,482.0,482.0,365.0,24.33,2,9


In [14]:
print(len(glob('data/video/clean_copy2/*')))

261


In [15]:
# separating the target
X = video_dataset
y = video_dataset['disease']

# creating the training and validation set
X_training, X_test, y_training, y_test = train_test_split(X, y, test_size=train_test_ratio, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_training, y_training, shuffle=True, test_size=train_val_ratio, stratify = y_training)

X_train = X_train.reset_index(drop = True)
X_val = X_val.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_val = y_val.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [16]:
print("X_train size : {}".format(X_train.shape))
print("y_train size : {}".format(y_train.shape))
print("X_val size : {}".format(X_val.shape))
print("y_val size : {}".format(y_val.shape))
print("X_test size : {}".format(X_test.shape))
print("y_test size : {}".format(y_test.shape))

X_train size : (78, 8)
y_train size : (78,)
X_val size : (20, 8)
y_val size : (20,)
X_test size : (43, 8)
y_test size : (43,)


test 영상만 이후 evaluate 하는 과정에서 frame을 추출하므로 파일 이름을 csv 파일로 저장

In [17]:
# storing the frames from training videos
test_dataset = pd.DataFrame()
test_dataset['filename'] = X_test['filename']
test_dataset.to_csv('data/test_dataset.csv', header=True, index=False)

train 영상과 val 영상은 지금 frame을 추출

다음 과정에서 생성되는 폴더는 코드를 다시 실행하는 경우 중복이나 필요없는 데이터를 제거하기 위해 이 폴더를 삭제하고 진행 (다음 코드 참고)

In [18]:
# create images_14frames_train and images_14frames_val folders, if they do not exist
if not os.path.exists('data/images_captured_train'):
    os.makedirs('data/images_captured_train')
if not os.path.exists('data/images_captured_val'):
    os.makedirs('data/images_captured_val')

files = glob('data/images_captured_train/*')
for f in files:
    os.remove(f)
files = glob('data/images_captured_val/*')
for f in files:
    os.remove(f)

Converts a PIL Image instance to a Numpy array. 
(tf.keras.image.img_to_array와 동일)

In [19]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
	  (im_height, im_width, channels)).astype(np.uint8)
   
def featuring_space(frame_count):
  if frame_count<=28:
    return 1
  elif frame_count<=56:
    return 2
  elif frame_count<=84:
    return 3
  elif frame_count<=112:
    return 4
  else : return 5

**모든 영상에서 뽑아내는 frame 길이를 동일하게 맞춘 뒤에 LSTM을 통과시키고자**

**모든 영상에 대해 frames에 해당하는 프레임을 capture함**

In [20]:
train_label=[]
for i in tqdm(range(X_train.shape[0])):

    count = 0
    current = 0
    videoFile = X_train['filename'][i]
    disease = videoFile.split('_')[2]
    if disease == 'covid' :
      disease_num = 0
    elif disease == 'pneumonia' :
      disease_num = 1
    else : disease_num = 2
    cap = cv2.VideoCapture('data/video/clean_copy2/' + videoFile) # capturing the video from the given path
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    space = featuring_space(frame_count)
    capture_epoch = math.floor( frame_count / (14*space) )
    capture_num = capture_epoch*14

    while(cap.isOpened()):
      if(count==capture_num):
        break
      cap.get(1)
      ret, frame = cap.read()
      if (ret != True):
        break
      if current%space==0 :
        filename = 'data/images_captured_train/' + videoFile + "_frame_%d.jpg" % count; count += 1
        cv2.imwrite(filename, frame)
        if(count%14==0):
          train_label.append(disease_num)
      current=current+1
    cap.release()

len(os.listdir('data/images_captured_train/'))

100%|██████████| 78/78 [00:43<00:00,  1.80it/s]


2114

In [21]:
val_label=[]
for i in tqdm(range(X_val.shape[0])):

    count = 0
    current = 0
    videoFile = X_val['filename'][i]
    disease = videoFile.split('_')[2]
    if disease == 'covid' :
      disease_num = 0
    elif disease == 'pneumonia' :
      disease_num = 1
    else : disease_num = 2
    cap = cv2.VideoCapture('data/video/clean_copy2/' + videoFile) # capturing the video from the given path
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    space = featuring_space(frame_count)
    capture_epoch = math.floor( frame_count / (14*space) )
    capture_num = capture_epoch*14

    while(cap.isOpened()):
      if(count==capture_num):
        break
      cap.get(1)
      ret, frame = cap.read()
      if (ret != True):
        break
      if current%space==0 :
        filename = 'data/images_captured_val/' + videoFile + "_frame_%d.jpg" % count; count += 1
        cv2.imwrite(filename, frame)
        if(count%14==0):
          val_label.append(disease_num)
      current=current+1
    cap.release()

len(os.listdir('data/images_captured_val/'))

100%|██████████| 20/20 [00:07<00:00,  2.86it/s]


434

In [22]:
X_val

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,16_butterfly_covid_prc_convex_clean.avi,15.0,634.0,634.0,202.0,13.47,0,1
1,61_pocusatlas_covid_prc_convex_clean.avi,15.0,472.0,472.0,21.0,1.40,0,4
2,232_clarius_pneumonia_prc_convex_clean.avi,15.0,458.0,458.0,482.0,32.13,1,9
3,239_paper_normal_prc_convex_clean.avi,15.0,512.0,512.0,197.0,13.13,2,8
4,72_pocusatlas_pneumonia_prc_convex_clean.avi,15.0,414.0,414.0,36.0,2.40,1,4
5,20_butterfly_normal_prc_convex_clean.avi,15.0,594.0,594.0,142.0,9.47,2,1
6,15_butterfly_covid_prc_convex_clean.avi,15.0,1054.0,1054.0,87.0,5.80,0,1
7,221_clarius_covid_prc_convex_clean.avi,15.0,400.0,400.0,83.0,5.53,0,9
8,42_grepmed_covid_prc_convex_clean.avi,15.0,434.0,434.0,159.0,10.60,0,2
9,26_grepmed_covid_prc_convex_clean.avi,15.0,382.0,382.0,70.0,4.67,0,2


In [23]:
train_images = glob("data/images_captured_train/*.jpg")

train_image_name = []
train_image_class = []
for i in tqdm(range(len(train_images))):
    # creating the image name
    train_image_name.append(train_images[i].split('/')[2])
    # creating the class of image
    disease = train_images[i].split('/')[2].split('_')[2]
    if disease == 'covid':
        train_image_class.append(0)
    elif disease == 'pneumonia':
        train_image_class.append(1)
    else:
        train_image_class.append(2)

train_image_dataset = pd.DataFrame()
train_image_dataset['filename'] = train_image_name
train_image_dataset['disease'] = train_image_class # 

train_image_dataset.to_csv('data/image_files_clean_train.csv', header=True, index=False)

100%|██████████| 2114/2114 [00:00<00:00, 569623.45it/s]


In [24]:
train_image_dataset

,filename,disease
0,94_butterfly_covid_prc_convex_clean.avi_frame_...,0
1,94_butterfly_covid_prc_convex_clean.avi_frame_...,0
2,94_butterfly_covid_prc_convex_clean.avi_frame_...,0
3,94_butterfly_covid_prc_convex_clean.avi_frame_...,0
4,94_butterfly_covid_prc_convex_clean.avi_frame_...,0
...,...,...
2109,204_paper_covid_prc_convex_clean.avi_frame_37.jpg,0
2110,204_paper_covid_prc_convex_clean.avi_frame_38.jpg,0
2111,204_paper_covid_prc_convex_clean.avi_frame_39.jpg,0
2112,204_paper_covid_prc_convex_clean.avi_frame_40.jpg,0


In [25]:
val_images = glob("data/images_captured_val/*.jpg")

val_image_name = []
val_image_class = []
for i in tqdm(range(len(val_images))):
    # creating the image name
    val_image_name.append(val_images[i].split('/')[2])
    # creating the class of image
    disease = val_images[i].split('/')[2].split('_')[2]
    if disease == 'covid':
        val_image_class.append(0)
    elif disease == 'pneumonia':
      val_image_class.append(1)
    else:
        val_image_class.append(2)

val_image_dataset = pd.DataFrame()
val_image_dataset['filename'] = val_image_name
val_image_dataset['disease'] = val_image_class 

val_image_dataset.to_csv('data/image_files_clean_val.csv', header=True, index=False)

100%|██████████| 434/434 [00:00<00:00, 505646.65it/s]


In [26]:
val_image_dataset

,filename,disease
0,16_butterfly_covid_prc_convex_clean.avi_frame_...,0
1,16_butterfly_covid_prc_convex_clean.avi_frame_...,0
2,16_butterfly_covid_prc_convex_clean.avi_frame_...,0
3,16_butterfly_covid_prc_convex_clean.avi_frame_...,0
4,16_butterfly_covid_prc_convex_clean.avi_frame_...,0
...,...,...
429,56_pocusatlas_covid_prc_convex_clean.avi_frame...,0
430,56_pocusatlas_covid_prc_convex_clean.avi_frame...,0
431,56_pocusatlas_covid_prc_convex_clean.avi_frame...,0
432,56_pocusatlas_covid_prc_convex_clean.avi_frame...,0


In [28]:
# creating an empty list
train_images = []

# for loop to read and store frames
for i in tqdm(range(train_image_dataset.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('data/images_captured_train/'+train_image_dataset['filename'][i], target_size=(height,width,channels))

    # converting it to array
    img_np = image.img_to_array(img)

    # normalizing the pixel value
    img_np = img_np / 255

    # appending the image to the images list
    train_images.append(img_np)
    
# converting the list to numpy array
X_train = np.array(train_images)
print(X_train.shape)

100%|██████████| 2114/2114 [00:16<00:00, 128.25it/s]


(2114, 224, 224, 3)


In [29]:
# creating an empty list
val_images = []

# for loop to read and store frames
for i in tqdm(range(val_image_dataset.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('data/images_captured_val/'+val_image_dataset['filename'][i], target_size=(height,width,channels))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the images list
    val_images.append(img)
    
# converting the list to numpy array
X_val = np.array(val_images)
print(X_val.shape)

100%|██████████| 434/434 [00:02<00:00, 164.41it/s]


(434, 224, 224, 3)


In [30]:
X_train=X_train.reshape(-1,14,224,224,3)
X_val=X_val.reshape(-1,14,224,224,3)
train_label = np.array(train_label)
val_label = np.array(val_label)
print(X_train.shape)
print(X_val.shape)
print(train_label.shape)
print(val_label.shape)
print(X_test.shape)

(151, 14, 224, 224, 3)
(31, 14, 224, 224, 3)
(151,)
(31,)
(43, 8)


In [31]:
np.save('data/X_train_save_0', X_train)
np.save('data/X_val_save_0', X_val)
np.save('data/y_train_save_0', train_label)
np.save('data/y_val_save_0', val_label)